# Homework 3 Function-based RL
Ponwalai Chalermwattanatrai 65340500042

## Part 1: Understanding the Algorithm

For each algorithm, describe whether it follows a value-based, policy-based, or Actor-Critic approach, specify the type of policy it learns (stochastic or deterministic), identify the type of observation space and action space (discrete or continuous), and explain how each advanced RL method balances exploration and exploitation.

#### 1.Linear Q-Learning

- **Approach**: Value-based
- **Policy type**: Deterministic (Stochastic during training)
    - Always chooses the action with the maximum Q-value (argmax Q(s, a)) -> Deterministic
    - During training, using epsilon-greedy, which includes some random -> Stochastic
- **Observation space**: continuous
- **Action space**: discrete
- **Balances exploration and exploitation**: using epsilon-greedy
    - Selects a random action with probability epsilon and decay epsilon over time